In [1]:
using Pkg
Pkg.activate("../Tambo/")

  Activating project at `~/research/TAMBO-MC/Tambo`


In [3]:
using Tambo
using StatsBase
using Distributions
using JLD2
using Plots
using Glob

In [7]:
config_f = jldopen("../resources/example_config.jld2")
const config = SimulationConfig(; Dict(k=>v for (k, v) in config_f["config"] if k != :geo_spline_path)...)
const geo = Tambo.Geometry(config)
const plane = Tambo.Plane(whitepaper_normal_vec, whitepaper_coord, geo)
const altmin = 1.8925255158436627units.km
const altmax = 4.092525515843662units.km

2.0739818168780746e10

In [5]:
fs = glob("event_dicts_00000_00???.jld2", "/Users/jlazar/Downloads/")

398-element Vector{String}:
 "/Users/jlazar/Downloads/event_dicts_00000_00001.jld2"
 "/Users/jlazar/Downloads/event_dicts_00000_00002.jld2"
 "/Users/jlazar/Downloads/event_dicts_00000_00003.jld2"
 "/Users/jlazar/Downloads/event_dicts_00000_00004.jld2"
 "/Users/jlazar/Downloads/event_dicts_00000_00005.jld2"
 "/Users/jlazar/Downloads/event_dicts_00000_00006.jld2"
 "/Users/jlazar/Downloads/event_dicts_00000_00007.jld2"
 "/Users/jlazar/Downloads/event_dicts_00000_00008.jld2"
 "/Users/jlazar/Downloads/event_dicts_00000_00009.jld2"
 "/Users/jlazar/Downloads/event_dicts_00000_00010.jld2"
 "/Users/jlazar/Downloads/event_dicts_00000_00011.jld2"
 "/Users/jlazar/Downloads/event_dicts_00000_00012.jld2"
 "/Users/jlazar/Downloads/event_dicts_00000_00013.jld2"
 ⋮
 "/Users/jlazar/Downloads/event_dicts_00000_00389.jld2"
 "/Users/jlazar/Downloads/event_dicts_00000_00390.jld2"
 "/Users/jlazar/Downloads/event_dicts_00000_00391.jld2"
 "/Users/jlazar/Downloads/event_dicts_00000_00392.jld2"
 "/Users/jlazar/D

In [8]:
key = "2000.0_100.0"

array_length = parse(Float64, split(key, '_')[1])
spacing = parse(Float64, split(key, '_')[2])

modules = Tambo.make_detector_array(
    whitepaper_coord,
    array_length * units.m,
    spacing * units.m,
    altmin,
    altmax,
    plane,
    geo
);

In [9]:
include("../scripts/trigger_defs.jl")

corsika_int_weight (generic function with 1 method)

In [18]:
weights_all = []
for f in fs
    
    
    # Load in hitmap
    hit_maps = nothing
    jldopen(f) do jldf
        hit_maps = jldf[key]
    
        # figure out which events triggered
        triggered_events = Int[]
        for event_id in keys(hit_maps)
            hit_map = hit_maps[event_id]
            if ~did_trigger(hit_map)
                continue
            end
            push!(triggered_events, parse(Int, event_id))
        end
        # Get the weights
        subrun_idx = parse(Int, split(split(f, ".")[end-1], "_")[end])
        jldopen("/Users/jlazar/Downloads/event_weights.jld2") do jldf
            weights = jldf[string(subrun_idx)][triggered_events]
        end

        weights_all = hcat(weights_all, weights)
    end
end

LoadError: UndefVarError: `weights_all` not defined